In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x65afba0


In [2]:
# TODO: make it possible to fit multiple runs simultaneously
run = {
    "lumi": 5000,
    "e_pol": 0.,
    "p_pol": 0.,
}

In [3]:
parameters = {
    "g1z": 0.0,
    "ka": 0.0,
    "la": 0.0,
}
obs_names = [
    "O_g1z_pos_1em05",
    "O_ka_pos_1em05",
    "O_la_pos_1em05",
]
input_path = "data/histograms/full/raw_histograms.root"
signal_cat = "4f_sw_sl_signal"
signal_processes = [
    "4f_sw_sl_eLpL_signal",
    "4f_sw_sl_eLpR_signal",
    "4f_sw_sl_eRpL_signal",
    "4f_sw_sl_eRpR_signal",
    ]
background_processes = []

In [4]:
signal_histograms = {}
signal_meta = {}
oo_matrix = {}
template_parameters = {}
with ROOT.TFile(input_path) as input_file:
    # take care of signals first
    signal_dir = input_file[signal_cat]
    for process_name in signal_processes:
        signal_histos = {}
        p_dir = signal_dir[process_name]
        for obs in obs_names:
            # very consistent ownership model of root requires us to do this
            histo = p_dir[obs]
            histo.SetDirectory(ROOT.nullptr)
            signal_histos[obs] = histo
            meta_dir = p_dir["meta"]
            obs_meta = {}
            for key in meta_dir.GetListOfKeys():
                key_name = key.GetName()
                obs_meta[key_name] = meta_dir[key_name]
        signal_histograms[process_name] = signal_histos
        signal_meta[process_name] = obs_meta
        # get OO matrix
        mat_dir = input_file["oo_matrix"]
        oo_matrix[process_name] = mat_dir[process_name]
    # get template parameters
    # unfortunately here we have the loops the other way around
    template_dir = input_file["template_parametrisations"]
    for obs in obs_names:
        obs_dir = template_dir[obs]
        p_pars = {}
        for process_name in signal_processes:
            p_dir = obs_dir[process_name]
            pars = {}
            for par in parameters.keys():
            # very consistent ownership model of root requires us to do
                par_hist = p_dir[par]
                par_hist.SetDirectory(ROOT.nullptr)
                pars[process_name] = par_hist
            p_pars[process_name] = pars
        template_parameters[obs] = p_pars


    # TODO: implement background handling

print(signal_histograms)
print(signal_meta)
print(oo_matrix)
print(template_parameters)

{'4f_sw_sl_eLpL_signal': {'O_g1z_pos_1em05': <cppyy.gbl.TH1D object at 0x85de610>, 'O_ka_pos_1em05': <cppyy.gbl.TH1D object at 0x87dbe00>, 'O_la_pos_1em05': <cppyy.gbl.TH1D object at 0x87b82b0>}, '4f_sw_sl_eLpR_signal': {'O_g1z_pos_1em05': <cppyy.gbl.TH1D object at 0x88a07b0>, 'O_ka_pos_1em05': <cppyy.gbl.TH1D object at 0x87889e0>, 'O_la_pos_1em05': <cppyy.gbl.TH1D object at 0x88e01f0>}, '4f_sw_sl_eRpL_signal': {'O_g1z_pos_1em05': <cppyy.gbl.TH1D object at 0x88f3250>, 'O_ka_pos_1em05': <cppyy.gbl.TH1D object at 0x88d88c0>, 'O_la_pos_1em05': <cppyy.gbl.TH1D object at 0x88b6f80>}, '4f_sw_sl_eRpR_signal': {'O_g1z_pos_1em05': <cppyy.gbl.TH1D object at 0x88f9400>, 'O_ka_pos_1em05': <cppyy.gbl.TH1D object at 0x88adc50>, 'O_la_pos_1em05': <cppyy.gbl.TH1D object at 0x88fc2d0>}}
{'4f_sw_sl_eLpL_signal': {'lumi': <cppyy.gbl.TParameter<float> object at 0x8803b70>, 'e_pol': <cppyy.gbl.TParameter<float> object at 0x87f7530>, 'p_pol': <cppyy.gbl.TParameter<float> object at 0x88199c0>}, '4f_sw_sl_eLp

In [ ]:
# try to write down the chi2 definition
# mean or sum?? just reference the part from Markus here
# sum! see (58) in https://inspirehep.net/literature/360476
# I will need one per run
# Jonas said it will be fine to make it just one big function, the automatic differentiation will take care of it then...